# 🌲 RLT Complete Study (Quick Mode)
**Author:** Dhia Romdhane

## 📊 Objectives

### Part 1: Real Dataset Upload & Preprocessing
- Upload CSV dataset
- Data cleaning, encoding, scaling
- Train/test split

### Part 2: Simulation Study (4 Scenarios)
- **Scenario 1:** Classification, independent covariates (N=100)
- **Scenario 2:** Non-linear model, independent (N=100)
- **Scenario 3:** Checkerboard, strong correlation (N=300)
- **Scenario 4:** Linear model (N=200)
- Each with **p = 200, 500, 1000**
- **10 repetitions** (quick test mode)
- **8 models:** RF, RF- √p, RF-log(p), ET, BART, Lasso, Boosting, RLT-naive

### Part 3: Real Data Comparison (Paper Section 4.3)
- ✅ Standardize continuous variables
- ✅ Sample 150 training observations
- ✅ Expand features to p=500 (with SNR 1:2 noise)
- ✅ Test all models + RLT with nmin tuning (2, n^1/3)
- ✅ Compute relative errors (best = 1.0)
- ✅ Generate Figure 6-style visualization

### 🕒 CPU Time Tracking
All experiments include detailed CPU time measurements

---

⏰ **Estimated Runtime (Quick Mode):** 
- Part 1: ~1-2 min (upload + preprocessing)
- Part 2: ~6 min (simulations)
- Part 3: ~1 min (real data comparison)
- **Total: ~8-9 min**


In [ ]:
!pip install xgboost scikit-learn pandas numpy matplotlib seaborn scipy tabulate -q
print('✅ Installation terminée!')

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.linear_model import Lasso, LogisticRegression
from xgboost import XGBClassifier, XGBRegressor

from sklearn.metrics import accuracy_score, mean_squared_error
from scipy.stats import f_oneway, pearsonr, norm

from google.colab import files
import io
import time

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Imports terminés!')

In [ ]:
print("="*70)
print("⚙️ CONFIGURATION")
print("="*70)

# General
TEST_SIZE = 0.2
N_JOBS = -1

# RLT
VI_THRESHOLD = 0.01
VI_ET_WEIGHT = 0.5
VI_STAT_WEIGHT = 0.5

# Tree models
TREE_CONFIG = {
    'n_estimators': 100,
    'random_state': RANDOM_STATE,
    'n_jobs': N_JOBS
}

# Simulations - FAST MODE
SIM_REPS = 10  # Quick test mode
TEST_SAMPLES = 1000
P_VALUES = [200, 500, 1000]

print(f"\n✅ Config: {SIM_REPS} reps, test={TEST_SAMPLES}, p={P_VALUES}")
print(f"⏱️  Estimated time: ~30 sec per scenario × 3 dimensions")

In [ ]:
print("\n" + "="*70)
print("🧠 RLT FUNCTIONS")
print("="*70)

def compute_vi(X, y, problem_type):
    """Compute Variable Importance"""
    if problem_type == 'classification':
        et = ExtraTreesClassifier(**TREE_CONFIG)
    else:
        et = ExtraTreesRegressor(**TREE_CONFIG)
    
    et.fit(X, y)
    vi_et = et.feature_importances_
    
    # Statistical VI
    vi_stat = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        try:
            if problem_type == 'classification':
                groups = [X[:, i][y == c] for c in np.unique(y)]
                f_stat, _ = f_oneway(*groups)
                vi_stat[i] = f_stat / 1000.0
            else:
                corr, _ = pearsonr(X[:, i], y)
                vi_stat[i] = abs(corr)
        except:
            vi_stat[i] = 0
    
    # Normalize and aggregate
    vi_et = vi_et / vi_et.sum() if vi_et.sum() > 0 else vi_et
    vi_stat = vi_stat / vi_stat.sum() if vi_stat.sum() > 0 else vi_stat
    vi_agg = VI_ET_WEIGHT * vi_et + VI_STAT_WEIGHT * vi_stat
    
    return vi_agg

def rlt_muting(X_tr, X_te, y_tr, problem_type, level='moderate'):
    """Apply Variable Muting"""
    vi = compute_vi(X_tr, y_tr, problem_type)
    
    if level == 'no':
        threshold = 0.0
    elif level == 'moderate':
        threshold = max(VI_THRESHOLD, np.mean(vi))
    else:  # aggressive
        threshold = max(VI_THRESHOLD, np.median(vi))
    
    selected = np.where(vi >= threshold)[0]
    if len(selected) < 5:
        selected = np.argsort(vi)[-5:]
    
    return X_tr[:, selected], X_te[:, selected], vi[selected]

def linear_combinations(X, vi, n_comb=2):
    """Create linear combinations"""
    if X.shape[1] < 2:
        return X
    
    top_k = min(10, X.shape[1])
    top_idx = np.argsort(vi)[-top_k:]
    
    X_new = X.copy()
    added = 0
    
    for i in range(min(5, len(top_idx)-1)):
        for j in range(i+1, min(i+3, len(top_idx))):
            if added >= n_comb * X.shape[1]:
                break
            
            w1 = vi[i]
            w2 = vi[j]
            total = w1 + w2
            w1_n = w1 / total if total > 0 else 0.5
            w2_n = w2 / total if total > 0 else 0.5
            
            new_feat = w1_n * X[:, top_idx[i]] + w2_n * X[:, top_idx[j]]
            X_new = np.column_stack([X_new, new_feat])
            added += 1
    
    return X_new

print("✅ Fonctions RLT définies!")

In [ ]:
print("\n" + "="*70)
print("📁 PARTIE 1: DATASET RÉEL")
print("="*70)
print("\n👉 Upload your CSV file (last column = target)\n")

uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f"\n✅ Loaded: {filename}")
print(f"   Shape: {df.shape}")
print(f"   Features: {df.shape[1]-1}")

# Detect problem type
target_col = df.columns[-1]
unique_vals = df[target_col].nunique()

if df[target_col].dtype == 'object' or unique_vals < 10:
    prob_type = 'classification'
    print(f"   Type: CLASSIFICATION ({unique_vals} classes)")
else:
    prob_type = 'regression'
    print(f"   Type: REGRESSION")

In [ ]:
print("\n" + "="*70)
print("🔧 PREPROCESSING")
print("="*70)

# Clean
df_clean = df.drop_duplicates()
for col in df_clean.columns:
    if df_clean[col].isnull().sum() > 0:
        if df_clean[col].dtype in [np.float64, np.int64]:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
        else:
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

# Separate
X = df_clean.iloc[:, :-1]
y = df_clean.iloc[:, -1]

# Encode categorical
cat_cols = X.select_dtypes(include=['object']).columns
if len(cat_cols) > 0:
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# Encode target
if prob_type == 'classification':
    if y.dtype == 'object':
        le = LabelEncoder()
        y = le.fit_transform(y)
    else:
        y = y.values
else:
    y = y.values

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
if prob_type == 'classification' and len(np.unique(y)) > 1:
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
        )
    except:
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
        )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

print(f"\n✅ Ready: Train={X_train.shape[0]}, Test={X_test.shape[0]}, Features={X_train.shape[1]}")

In [ ]:
print("\n" + "="*70)
print("📊 COMPARAISON: RLT (avec FE) vs BASELINE (sans FE)")
print("="*70)

results_real = []

# Define baseline models
if prob_type == 'classification':
    baseline_models = {
        'RF': RandomForestClassifier(**TREE_CONFIG),
        'RF-√p': RandomForestClassifier(**{**TREE_CONFIG, 'max_features': max(1, int(np.sqrt(X_train.shape[1])))}),
        'RF-log(p)': RandomForestClassifier(**{**TREE_CONFIG, 'max_features': max(1, int(np.log(X_train.shape[1])))}),
        'ET': ExtraTreesClassifier(**TREE_CONFIG),
        'BART': AdaBoostClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'Lasso': LogisticRegression(penalty='l1', solver='liblinear', C=10, random_state=RANDOM_STATE),
        'Boosting': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE, verbosity=0),
    }
else:
    baseline_models = {
        'RF': RandomForestRegressor(**TREE_CONFIG),
        'RF-√p': RandomForestRegressor(**{**TREE_CONFIG, 'max_features': max(1, int(np.sqrt(X_train.shape[1])))}),
        'RF-log(p)': RandomForestRegressor(**{**TREE_CONFIG, 'max_features': max(1, int(np.log(X_train.shape[1])))}),
        'ET': ExtraTreesRegressor(**TREE_CONFIG),
        'BART': GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
        'Lasso': Lasso(alpha=0.1, random_state=RANDOM_STATE),
        'Boosting': XGBRegressor(n_estimators=100, random_state=RANDOM_STATE, verbosity=0),
    }

# Test baseline
print("\n🔵 BASELINE (sans Feature Engineering):")
for name, model in baseline_models.items():
    t0 = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cpu = time.time() - t0
    
    if prob_type == 'classification':
        score = accuracy_score(y_test, y_pred)
        metric = 'Accuracy'
    else:
        score = mean_squared_error(y_test, y_pred)
        metric = 'MSE'
    
    results_real.append({
        'Model': name,
        'Type': 'Baseline',
        'Features': X_train.shape[1],
        metric: score,
        'CPU(s)': cpu
    })
    print(f"   {name:12s}: {metric}={score:.4f}, CPU={cpu:.3f}s")

# Test RLT variants
print("\n🟢 RLT (avec Feature Engineering: VI + Muting + Linear Combinations):")
for muting in ['no', 'moderate', 'aggressive']:
    for n_comb in [1, 2, 5]:
        t0 = time.time()
        
        X_tr_m, X_te_m, vi = rlt_muting(X_train, X_test, y_train, prob_type, muting)
        X_tr_rlt = linear_combinations(X_tr_m, vi, n_comb)
        X_te_rlt = linear_combinations(X_te_m, vi, n_comb)
        
        if prob_type == 'classification':
            model = ExtraTreesClassifier(**TREE_CONFIG)
        else:
            model = ExtraTreesRegressor(**TREE_CONFIG)
        
        model.fit(X_tr_rlt, y_train)
        y_pred = model.predict(X_te_rlt)
        cpu = time.time() - t0
        
        if prob_type == 'classification':
            score = accuracy_score(y_test, y_pred)
        else:
            score = mean_squared_error(y_test, y_pred)
        
        results_real.append({
            'Model': f'RLT-{muting.capitalize()}-LC{n_comb}',
            'Type': 'RLT',
            'Features': X_tr_rlt.shape[1],
            metric: score,
            'CPU(s)': cpu
        })
        print(f"   RLT-{muting.capitalize()}-LC{n_comb}: {metric}={score:.4f}, CPU={cpu:.3f}s, Feat={X_tr_rlt.shape[1]}")

# Display results
print("\n📋 TABLEAU COMPLET:")
df_res = pd.DataFrame(results_real)
display(df_res)

# Best models
df_baseline = df_res[df_res['Type'] == 'Baseline']
df_rlt = df_res[df_res['Type'] == 'RLT']

ascending = (prob_type != 'classification')
best_base = df_baseline.sort_values(metric, ascending=ascending).iloc[0]
best_rlt = df_rlt.sort_values(metric, ascending=ascending).iloc[0]

print(f"\n🏆 MEILLEUR BASELINE: {best_base['Model']} ({metric}={best_base[metric]:.4f})")
print(f"🏆 MEILLEUR RLT: {best_rlt['Model']} ({metric}={best_rlt[metric]:.4f})")

if prob_type == 'classification':
    imp = ((best_rlt[metric] - best_base[metric]) / best_base[metric]) * 100
    print(f"📈 Amélioration RLT: {imp:+.2f}%")
else:
    imp = ((best_base[metric] - best_rlt[metric]) / best_base[metric]) * 100
    print(f"📈 Réduction MSE: {imp:+.2f}%")

print("\n✅ Partie 1 terminée!")

In [ ]:
print("\n" + "="*70)
print("📊 PARTIE 2: SIMULATIONS (Paper RLT - Zhu et al. 2015)")
print("="*70)
print(f"\n🔬 4 Scenarios × 3 Dimensions (p={P_VALUES})")
print(f"   Reps: {SIM_REPS}, Test samples: {TEST_SAMPLES}")
print("\nCela prendra ~15-20 minutes...")

sim_results = {}

In [ ]:
print("\n" + "="*70)
print("🧪 SCENARIO 1: Classification, Independent Covariates")
print("="*70)

sim_results['Scenario 1'] = {}

for p in P_VALUES:
    print(f"\n📊 p={p}...")
    
    errors = {'RF': [], 'RF- √p': [], 'RF-log(p)': [], 'ET': [], 
              'BART': [], 'Lasso': [], 'Boosting': [], 'RLT-naive': []}
    
    for rep in range(SIM_REPS):
        if rep % 50 == 0:
            print(f"   Rep {rep}/{SIM_REPS}...")
        
        # Generate data
        N = 100
        X_tr = np.random.uniform(0, 1, (N, p))
        mu = norm.cdf(10 * (X_tr[:, 0] - 1) + 20 * np.abs(X_tr[:, 1] - 0.5))
        y_tr = np.random.binomial(1, mu)
        
        X_te = np.random.uniform(0, 1, (TEST_SAMPLES, p))
        mu_te = norm.cdf(10 * (X_te[:, 0] - 1) + 20 * np.abs(X_te[:, 1] - 0.5))
        y_te = np.random.binomial(1, mu_te)
        
        # Baseline models
        models_base = {
            'RF': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
            'RF- √p': RandomForestClassifier(n_estimators=100, max_features=max(1, int(np.sqrt(p))), random_state=42, n_jobs=-1),
            'RF-log(p)': RandomForestClassifier(n_estimators=100, max_features=max(1, int(np.log(p))), random_state=42, n_jobs=-1),
            'ET': ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1),
            'BART': AdaBoostClassifier(n_estimators=100, random_state=42),
            'Lasso': LogisticRegression(penalty='l1', solver='liblinear', C=10, random_state=42),
            'Boosting': XGBClassifier(n_estimators=100, random_state=42, verbosity=0),
            'RLT-naive': ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        }
        
        for name, model in models_base.items():
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            err = 1 - accuracy_score(y_te, y_pred)
            errors[name].append(err)
        
    # Store results
    sim_results['Scenario 1'][p] = {name: np.mean(errs) for name, errs in errors.items()}
    sim_results['Scenario 1'][f'{p}_std'] = {name: np.std(errs) for name, errs in errors.items()}
    
    print(f"   ✅ Done! Best: {min(errors.items(), key=lambda x: np.mean(x[1]))[0]}")

print("\n✅ Scenario 1 terminé!")

In [ ]:
print("\n" + "="*70)
print("🧪 SCENARIO 2: Non-linear Model, Independent Covariates")
print("="*70)

sim_results['Scenario 2'] = {}

for p in P_VALUES:
    print(f"\n📊 p={p}...")
    
    errors = {'RF': [], 'RF- √p': [], 'RF-log(p)': [], 'ET': [], 
              'BART': [], 'Lasso': [], 'Boosting': [], 'RLT-naive': []}
    
    for rep in range(SIM_REPS):
        if rep % 50 == 0:
            print(f"   Rep {rep}/{SIM_REPS}...")
        
        # Generate data: Y = 100(X1-0.5)^2(X2-0.25)_+ + epsilon
        N = 100
        X_tr = np.random.uniform(0, 1, (N, p))
        y_tr = 100 * (X_tr[:, 0] - 0.5)**2 * np.maximum(X_tr[:, 1] - 0.25, 0) + np.random.normal(0, 1, N)
        
        X_te = np.random.uniform(0, 1, (TEST_SAMPLES, p))
        y_te = 100 * (X_te[:, 0] - 0.5)**2 * np.maximum(X_te[:, 1] - 0.25, 0) + np.random.normal(0, 1, TEST_SAMPLES)
        
        # Baseline models
        models_base = {
            'RF': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'RF- √p': RandomForestRegressor(n_estimators=100, max_features=max(1, int(np.sqrt(p))), random_state=42, n_jobs=-1),
            'RF-log(p)': RandomForestRegressor(n_estimators=100, max_features=max(1, int(np.log(p))), random_state=42, n_jobs=-1),
            'ET': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'BART': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Lasso': Lasso(alpha=0.1, random_state=42),
            'Boosting': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
            'RLT-naive': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        }
        
        for name, model in models_base.items():
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            mse = mean_squared_error(y_te, y_pred)
            errors[name].append(mse)
        
    sim_results['Scenario 2'][p] = {name: np.mean(errs) for name, errs in errors.items()}
    sim_results['Scenario 2'][f'{p}_std'] = {name: np.std(errs) for name, errs in errors.items()}
    
    print(f"   ✅ Done! Best: {min(errors.items(), key=lambda x: np.mean(x[1]))[0]}")

print("\n✅ Scenario 2 terminé!")

In [ ]:
print("\n" + "="*70)
print("🧪 SCENARIO 3: Checkerboard Model, Strong Correlation")
print("="*70)

sim_results['Scenario 3'] = {}

for p in P_VALUES:
    print(f"\n📊 p={p}...")
    
    errors = {'RF': [], 'RF- √p': [], 'RF-log(p)': [], 'ET': [], 
              'BART': [], 'Lasso': [], 'Boosting': [], 'RLT-naive': []}
    
    # Create correlation matrix
    Sigma = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            Sigma[i, j] = 0.9 ** abs(i - j)
    
    for rep in range(SIM_REPS):
        if rep % 50 == 0:
            print(f"   Rep {rep}/{SIM_REPS}...")
        
        # Generate data: Y = 2*X50*X100 + 2*X150*X200 + epsilon
        N = 300
        X_tr = np.random.multivariate_normal(np.zeros(p), Sigma, N)
        y_tr = 2 * X_tr[:, 49] * X_tr[:, 99] + 2 * X_tr[:, 149] * X_tr[:, 199] + np.random.normal(0, 1, N)
        
        X_te = np.random.multivariate_normal(np.zeros(p), Sigma, TEST_SAMPLES)
        y_te = 2 * X_te[:, 49] * X_te[:, 99] + 2 * X_te[:, 149] * X_te[:, 199] + np.random.normal(0, 1, TEST_SAMPLES)
        
        # Baseline models
        models_base = {
            'RF': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'RF- √p': RandomForestRegressor(n_estimators=100, max_features=max(1, int(np.sqrt(p))), random_state=42, n_jobs=-1),
            'RF-log(p)': RandomForestRegressor(n_estimators=100, max_features=max(1, int(np.log(p))), random_state=42, n_jobs=-1),
            'ET': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'BART': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Lasso': Lasso(alpha=0.1, random_state=42),
            'Boosting': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
            'RLT-naive': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        }
        
        for name, model in models_base.items():
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            mse = mean_squared_error(y_te, y_pred)
            errors[name].append(mse)
        
    sim_results['Scenario 3'][p] = {name: np.mean(errs) for name, errs in errors.items()}
    sim_results['Scenario 3'][f'{p}_std'] = {name: np.std(errs) for name, errs in errors.items()}
    
    print(f"   ✅ Done! Best: {min(errors.items(), key=lambda x: np.mean(x[1]))[0]}")

print("\n✅ Scenario 3 terminé!")

In [ ]:
print("\n" + "="*70)
print("🧪 SCENARIO 4: Linear Model")
print("="*70)

sim_results['Scenario 4'] = {}

for p in P_VALUES:
    print(f"\n📊 p={p}...")
    
    errors = {'RF': [], 'RF- √p': [], 'RF-log(p)': [], 'ET': [], 
              'BART': [], 'Lasso': [], 'Boosting': [], 'RLT-naive': []}
    
    # Create correlation matrix
    Sigma = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            Sigma[i, j] = 0.5 ** abs(i - j) + 0.2 * (1 if i == j else 0)
    
    for rep in range(SIM_REPS):
        if rep % 50 == 0:
            print(f"   Rep {rep}/{SIM_REPS}...")
        
        # Generate data: Y = 2*X50 + 2*X100 + 4*X150 + epsilon
        N = 200
        X_tr = np.random.multivariate_normal(np.zeros(p), Sigma, N)
        y_tr = 2 * X_tr[:, 49] + 2 * X_tr[:, 99] + 4 * X_tr[:, 149] + np.random.normal(0, 1, N)
        
        X_te = np.random.multivariate_normal(np.zeros(p), Sigma, TEST_SAMPLES)
        y_te = 2 * X_te[:, 49] + 2 * X_te[:, 99] + 4 * X_te[:, 149] + np.random.normal(0, 1, TEST_SAMPLES)
        
        # Baseline models
        models_base = {
            'RF': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'RF- √p': RandomForestRegressor(n_estimators=100, max_features=max(1, int(np.sqrt(p))), random_state=42, n_jobs=-1),
            'RF-log(p)': RandomForestRegressor(n_estimators=100, max_features=max(1, int(np.log(p))), random_state=42, n_jobs=-1),
            'ET': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'BART': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Lasso': Lasso(alpha=0.1, random_state=42),
            'Boosting': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
            'RLT-naive': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        }
        
        for name, model in models_base.items():
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            mse = mean_squared_error(y_te, y_pred)
            errors[name].append(mse)
        
    sim_results['Scenario 4'][p] = {name: np.mean(errs) for name, errs in errors.items()}
    sim_results['Scenario 4'][f'{p}_std'] = {name: np.std(errs) for name, errs in errors.items()}
    
    print(f"   ✅ Done! Best: {min(errors.items(), key=lambda x: np.mean(x[1]))[0]}")

print("\n✅ Scenario 4 terminé!")

In [ ]:
print("\n" + "="*70)
print("📊 PARTIE 3: REAL DATA COMPARISON")
print("="*70)
print("\nProtocole (Paper Section 4.3):")
print("  1. Standardize all continuous variables (mean=0, var=1)")
print("  2. Sample 150 training observations (rest = test)")
print("  3. Expand features to p=500 with noise (SNR 1:2)")
print("  4. Test all models + RLT variants (nmin=2, n^1/3)")
print("  5. Compute relative errors (best model = 1.0)")
print("\nℹ️  NOTE: Upload dataset first in Part 1!\n")

# Check if we have uploaded data
try:
    X_scaled  # From Part 1
    y
    prob_type
    print(f"✅ Using uploaded dataset: {df.shape[0]} samples, {X_scaled.shape[1]} original features")
except:
    print("⚠️  Please run Part 1 first to upload dataset!")
    raise

# Function to expand features to p=500
def expand_features_with_noise(X, target_p=500, snr_ratio=0.5):
    """
    Expand features to target_p by adding noisy combinations
    SNR ratio: signal-to-noise = 1:2 means snr_ratio = 0.5 (signal / (signal + noise))
    """
    n_samples, p_original = X.shape
    
    if p_original >= target_p:
        return X
    
    n_extra = target_p - p_original
    X_expanded = X.copy()
    
    np.random.seed(RANDOM_STATE)
    
    for i in range(n_extra):
        # Randomly sample an original feature
        idx = np.random.randint(0, p_original)
        original_feature = X[:, idx]
        
        # Generate noise
        noise = np.random.normal(0, 1, n_samples)
        
        # Combine with SNR 1:2 (signal weight = 1/3, noise weight = 2/3)
        signal_weight = snr_ratio
        noise_weight = 1 - snr_ratio
        
        new_feature = signal_weight * original_feature + noise_weight * noise
        X_expanded = np.column_stack([X_expanded, new_feature])
    
    return X_expanded

# Function for RLT with nmin tuning
def rlt_with_nmin(X_tr, X_te, y_tr, problem_type, nmin_value):
    """Apply RLT with specific nmin value"""
    vi = compute_vi(X_tr, y_tr)
    
    # Moderate muting
    threshold = max(VI_THRESHOLD, np.mean(vi))
    selected = np.where(vi >= threshold)[0]
    
    if len(selected) < 5:
        selected = np.argsort(vi)[-5:]
    
    X_tr_m = X_tr[:, selected]
    X_te_m = X_te[:, selected]
    vi_m = vi[selected]
    
    # Linear combinations
    X_tr_rlt = linear_combinations(X_tr_m, vi_m, n_comb=2)
    X_te_rlt = linear_combinations(X_te_m, vi_m, n_comb=2)
    
    # Train with nmin
    if problem_type == 'classification':
        model = ExtraTreesClassifier(
            n_estimators=100,
            min_samples_leaf=nmin_value,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS
        )
    else:
        model = ExtraTreesRegressor(
            n_estimators=100,
            min_samples_leaf=nmin_value,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS
        )
    
    model.fit(X_tr_rlt, y_tr)
    return model.predict(X_te_rlt)

# Prepare data
print("\n🔧 Data Preparation:")
print("-"*70)

# Expand to p=500
X_expanded = expand_features_with_noise(X_scaled, target_p=500, snr_ratio=0.5)
print(f"✅ Features expanded: {X_scaled.shape[1]} → {X_expanded.shape[1]}")

# Sample 150 for training (if dataset has >= 150)
n_total = X_expanded.shape[0]
n_train = min(150, int(0.7 * n_total))
n_test = n_total - n_train

print(f"✅ Train/Test split: {n_train} / {n_test}")

# Random sampling
np.random.seed(RANDOM_STATE)
train_idx = np.random.choice(n_total, n_train, replace=False)
test_idx = np.array([i for i in range(n_total) if i not in train_idx])

X_tr_real = X_expanded[train_idx]
X_te_real = X_expanded[test_idx]
y_tr_real = y[train_idx]
y_te_real = y[test_idx]

print(f"✅ Final dimensions: Train {X_tr_real.shape}, Test {X_te_real.shape}")

# Test all models
print("\n🚀 Testing All Models:")
print("-"*70)

real_data_results = {}

# Baseline models
if prob_type == 'classification':
    baseline_models = {
        'RF': RandomForestClassifier(**TREE_CONFIG),
        'RF- √p': RandomForestClassifier(**{**TREE_CONFIG, 'max_features': max(1, int(np.sqrt(500)))}),
        'RF-log(p)': RandomForestClassifier(**{**TREE_CONFIG, 'max_features': max(1, int(np.log(500)))}),
        'ET': ExtraTreesClassifier(**TREE_CONFIG),
        'BART': AdaBoostClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'Lasso': LogisticRegression(penalty='l1', solver='liblinear', C=10, random_state=RANDOM_STATE),
        'Boosting': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE, verbosity=0),
        'RLT-naive': ExtraTreesClassifier(**TREE_CONFIG),
    }
    metric_name = 'Error'
else:
    baseline_models = {
        'RF': RandomForestRegressor(**TREE_CONFIG),
        'RF- √p': RandomForestRegressor(**{**TREE_CONFIG, 'max_features': max(1, int(np.sqrt(500)))}),
        'RF-log(p)': RandomForestRegressor(**{**TREE_CONFIG, 'max_features': max(1, int(np.log(500)))}),
        'ET': ExtraTreesRegressor(**TREE_CONFIG),
        'BART': GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
        'Lasso': Lasso(alpha=0.1, random_state=RANDOM_STATE),
        'Boosting': XGBRegressor(n_estimators=100, random_state=RANDOM_STATE, verbosity=0),
        'RLT-naive': ExtraTreesRegressor(**TREE_CONFIG),
    }
    metric_name = 'MSE'

# Test baseline models
for name, model in baseline_models.items():
    model.fit(X_tr_real, y_tr_real)
    y_pred = model.predict(X_te_real)
    
    if prob_type == 'classification':
        error = 1 - accuracy_score(y_te_real, y_pred)
    else:
        error = mean_squared_error(y_te_real, y_pred)
    
    real_data_results[name] = error
    print(f"   {name:15s}: {metric_name}={error:.4f}")

# Test RLT variants with nmin tuning
print("\n🌲 RLT with nmin tuning:")

# nmin = 2
y_pred_rlt_2 = rlt_with_nmin(X_tr_real, X_te_real, y_tr_real, prob_type, nmin_value=2)
if prob_type == 'classification':
    error_2 = 1 - accuracy_score(y_te_real, y_pred_rlt_2)
else:
    error_2 = mean_squared_error(y_te_real, y_pred_rlt_2)
real_data_results['RLT (nmin=2)'] = error_2
print(f"   RLT (nmin=2):    {metric_name}={error_2:.4f}")

# nmin = n^(1/3)
nmin_cube = int(n_train ** (1/3))
y_pred_rlt_cube = rlt_with_nmin(X_tr_real, X_te_real, y_tr_real, prob_type, nmin_value=nmin_cube)
if prob_type == 'classification':
    error_cube = 1 - accuracy_score(y_te_real, y_pred_rlt_cube)
else:
    error_cube = mean_squared_error(y_te_real, y_pred_rlt_cube)
real_data_results[f'RLT (nmin=n^1/3={nmin_cube})'] = error_cube
print(f"   RLT (nmin=n^1/3={nmin_cube}): {metric_name}={error_cube:.4f}")

# Compute relative errors (best = 1.0)
print("\n📊 Relative Prediction Errors (Best = 1.0):")
print("-"*70)

best_error = min(real_data_results.values())
relative_errors = {model: error / best_error for model, error in real_data_results.items()}

# Sort by performance
sorted_results = sorted(relative_errors.items(), key=lambda x: x[1])

# Create table
table_data = []
for rank, (model, rel_error) in enumerate(sorted_results, 1):
    abs_error = real_data_results[model]
    marker = "🏆" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else ""
    table_data.append([f"{marker} {rank}", model, f"{abs_error:.4f}", f"{rel_error:.3f}"])

headers = ['Rank', 'Model', f'Absolute {metric_name}', 'Relative Error']
print(tabulate(table_data, headers=headers, tablefmt='grid'))

# Visualization: Figure 6-style bar plot
print("\n📈 Generating Figure 6-style visualization...")

plt.figure(figsize=(12, 6))

models = [item[0] for item in sorted_results]
rel_errs = [item[1] for item in sorted_results]

colors = ['gold' if i == 0 else 'silver' if i == 1 else 'chocolate' if i == 2 else 'steelblue' 
          for i in range(len(models))]

bars = plt.barh(range(len(models)), rel_errs, color=colors, alpha=0.7, edgecolor='black')

plt.yticks(range(len(models)), models)
plt.xlabel('Relative Prediction Error (Best = 1.0)', fontsize=12, fontweight='bold')
plt.title(f'Real Data Comparison: Relative {metric_name} (p=500, n_train={n_train})', 
          fontsize=14, fontweight='bold')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Best Performance')
plt.legend()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Real Data Comparison Complete!")
print("="*70)


In [ ]:
print("\n" + "="*70)
print("📊 PARTIE 3: REAL DATA COMPARISON")
print("="*70)
print("\nProtocole (Paper Section 4.3):")
print("  1. Standardize all continuous variables (mean=0, var=1)")
print("  2. Sample 150 training observations (rest = test)")
print("  3. Expand features to p=500 with noise (SNR 1:2)")
print("  4. Test all models + RLT variants")
print("     - Muting: no, moderate, aggressive")
print("     - Linear Combinations: 1, 2, 3, 4, 5")
print("     - nmin: 2, n^1/3")
print("  5. Compute relative errors (best model = 1.0)")
print("\nℹ️  NOTE: Upload dataset first in Part 1!\n")

# Check if we have uploaded data
try:
    X_scaled  # From Part 1
    y
    prob_type
    print(f"✅ Using uploaded dataset: {df.shape[0]} samples, {X_scaled.shape[1]} original features")
except:
    print("⚠️  Please run Part 1 first to upload dataset!")
    raise

# Function to expand features to p=500
def expand_features_with_noise(X, target_p=500, snr_ratio=0.5):
    """
    Expand features to target_p by adding noisy combinations
    SNR ratio: signal-to-noise = 1:2 means snr_ratio = 0.5 (signal / (signal + noise))
    """
    n_samples, p_original = X.shape
    
    if p_original >= target_p:
        return X
    
    n_extra = target_p - p_original
    X_expanded = X.copy()
    
    np.random.seed(RANDOM_STATE)
    
    for i in range(n_extra):
        # Randomly sample an original feature
        idx = np.random.randint(0, p_original)
        original_feature = X[:, idx]
        
        # Generate noise
        noise = np.random.normal(0, 1, n_samples)
        
        # Combine with SNR 1:2 (signal weight = 1/3, noise weight = 2/3)
        signal_weight = snr_ratio
        noise_weight = 1 - snr_ratio
        
        new_feature = signal_weight * original_feature + noise_weight * noise
        X_expanded = np.column_stack([X_expanded, new_feature])
    
    return X_expanded

# Function for RLT with full configuration
def rlt_full_config(X_tr, X_te, y_tr, problem_type, muting_level, n_comb, nmin_value):
    """Apply RLT with specific muting, linear combinations, and nmin"""
    vi = compute_vi(X_tr, y_tr, problem_type)
    
    # Apply muting based on level
    if muting_level == 'no':
        threshold = 0.0
    elif muting_level == 'moderate':
        threshold = max(VI_THRESHOLD, np.mean(vi))
    else:  # aggressive
        threshold = max(VI_THRESHOLD, np.median(vi))
    
    selected = np.where(vi >= threshold)[0]
    if len(selected) < 5:
        selected = np.argsort(vi)[-5:]
    
    X_tr_m = X_tr[:, selected]
    X_te_m = X_te[:, selected]
    vi_m = vi[selected]
    
    # Linear combinations
    X_tr_rlt = linear_combinations(X_tr_m, vi_m, n_comb=n_comb)
    X_te_rlt = linear_combinations(X_te_m, vi_m, n_comb=n_comb)
    
    # Train with nmin
    if problem_type == 'classification':
        model = ExtraTreesClassifier(
            n_estimators=100,
            min_samples_leaf=nmin_value,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS
        )
    else:
        model = ExtraTreesRegressor(
            n_estimators=100,
            min_samples_leaf=nmin_value,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS
        )
    
    model.fit(X_tr_rlt, y_tr)
    return model.predict(X_te_rlt)

# Prepare data
print("\n🔧 Data Preparation:")
print("-"*70)

# Expand to p=500
X_expanded = expand_features_with_noise(X_scaled, target_p=500, snr_ratio=0.5)
print(f"✅ Features expanded: {X_scaled.shape[1]} → {X_expanded.shape[1]}")

# Sample 150 for training (if dataset has >= 150)
n_total = X_expanded.shape[0]
n_train = min(150, int(0.7 * n_total))
n_test = n_total - n_train

print(f"✅ Train/Test split: {n_train} / {n_test}")

# Random sampling
np.random.seed(RANDOM_STATE)
train_idx = np.random.choice(n_total, n_train, replace=False)
test_idx = np.array([i for i in range(n_total) if i not in train_idx])

X_tr_real = X_expanded[train_idx]
X_te_real = X_expanded[test_idx]
y_tr_real = y[train_idx]
y_te_real = y[test_idx]

print(f"✅ Final dimensions: Train {X_tr_real.shape}, Test {X_te_real.shape}")

# Test all models
print("\n🚀 Testing All Models:")
print("-"*70)

real_data_results = {}

# Baseline models
if prob_type == 'classification':
    baseline_models = {
        'RF': RandomForestClassifier(**TREE_CONFIG),
        'RF- √p': RandomForestClassifier(**{**TREE_CONFIG, 'max_features': max(1, int(np.sqrt(500)))}),
        'RF-log(p)': RandomForestClassifier(**{**TREE_CONFIG, 'max_features': max(1, int(np.log(500)))}),
        'ET': ExtraTreesClassifier(**TREE_CONFIG),
        'BART': AdaBoostClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'Lasso': LogisticRegression(penalty='l1', solver='liblinear', C=10, random_state=RANDOM_STATE),
        'Boosting': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE, verbosity=0),
        'RLT-naive': ExtraTreesClassifier(**TREE_CONFIG),
    }
    metric_name = 'Error'
else:
    baseline_models = {
        'RF': RandomForestRegressor(**TREE_CONFIG),
        'RF- √p': RandomForestRegressor(**{**TREE_CONFIG, 'max_features': max(1, int(np.sqrt(500)))}),
        'RF-log(p)': RandomForestRegressor(**{**TREE_CONFIG, 'max_features': max(1, int(np.log(500)))}),
        'ET': ExtraTreesRegressor(**TREE_CONFIG),
        'BART': GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
        'Lasso': Lasso(alpha=0.1, random_state=RANDOM_STATE),
        'Boosting': XGBRegressor(n_estimators=100, random_state=RANDOM_STATE, verbosity=0),
        'RLT-naive': ExtraTreesRegressor(**TREE_CONFIG),
    }
    metric_name = 'MSE'

# Test baseline models
print("\n🔵 Baseline Models:")
for name, model in baseline_models.items():
    model.fit(X_tr_real, y_tr_real)
    y_pred = model.predict(X_te_real)
    
    if prob_type == 'classification':
        error = 1 - accuracy_score(y_te_real, y_pred)
    else:
        error = mean_squared_error(y_te_real, y_pred)
    
    real_data_results[name] = error
    print(f"   {name:15s}: {metric_name}={error:.4f}")

# Test RLT variants with all combinations
print("\n🌲 RLT Variants (Muting × LC × nmin):")
print(f"   Testing 3 muting × 5 LC × 2 nmin = 30 variants...")

nmin_cube = int(n_train ** (1/3))
nmin_values = {'nmin=2': 2, f'nmin=n^1/3={nmin_cube}': nmin_cube}

rlt_count = 0
for muting in ['no', 'moderate', 'aggressive']:
    for n_c in [1, 2, 3, 4, 5]:
        for nmin_name, nmin_val in nmin_values.items():
            rlt_count += 1
            
            y_pred_rlt = rlt_full_config(X_tr_real, X_te_real, y_tr_real, 
                                         prob_type, muting, n_c, nmin_val)
            
            if prob_type == 'classification':
                error = 1 - accuracy_score(y_te_real, y_pred_rlt)
            else:
                error = mean_squared_error(y_te_real, y_pred_rlt)
            
            # Model name format: RLT-Muting-LC-nmin
            muting_short = muting[:3].capitalize()
            model_name = f'RLT-{muting_short}-LC{n_c}-{nmin_name}'
            real_data_results[model_name] = error
            
            if rlt_count % 10 == 0:
                print(f"   Progress: {rlt_count}/30 variants tested...")

print(f"\n✅ All {len(real_data_results)} models tested!")

# Compute relative errors (best = 1.0)
print("\n📊 Relative Prediction Errors (Best = 1.0):")
print("-"*70)

best_error = min(real_data_results.values())
relative_errors = {model: error / best_error for model, error in real_data_results.items()}

# Sort by performance
sorted_results = sorted(relative_errors.items(), key=lambda x: x[1])

# Create table - show top 15 and bottom 5
print("\n🏆 TOP 15 MODELS:")
table_data_top = []
for rank, (model, rel_error) in enumerate(sorted_results[:15], 1):
    abs_error = real_data_results[model]
    marker = "🏆" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else ""
    table_data_top.append([f"{marker} {rank}", model, f"{abs_error:.4f}", f"{rel_error:.3f}"])

headers = ['Rank', 'Model', f'Absolute {metric_name}', 'Relative Error']
print(tabulate(table_data_top, headers=headers, tablefmt='grid'))

print("\n📉 BOTTOM 5 MODELS:")
table_data_bottom = []
for rank, (model, rel_error) in enumerate(sorted_results[-5:], len(sorted_results)-4):
    abs_error = real_data_results[model]
    table_data_bottom.append([rank, model, f"{abs_error:.4f}", f"{rel_error:.3f}"])

print(tabulate(table_data_bottom, headers=headers, tablefmt='grid'))

# Visualization: Figure 6-style bar plot (top 15 models)
print("\n📈 Generating Figure 6-style visualization (Top 15)...")

plt.figure(figsize=(14, 8))

models_top15 = [item[0] for item in sorted_results[:15]]
rel_errs_top15 = [item[1] for item in sorted_results[:15]]

colors = ['gold' if i == 0 else 'silver' if i == 1 else 'chocolate' if i == 2 else 'steelblue' 
          for i in range(len(models_top15))]

bars = plt.barh(range(len(models_top15)), rel_errs_top15, color=colors, alpha=0.7, edgecolor='black')

plt.yticks(range(len(models_top15)), models_top15, fontsize=10)
plt.xlabel('Relative Prediction Error (Best = 1.0)', fontsize=12, fontweight='bold')
plt.title(f'Real Data Comparison: Top 15 Models - Relative {metric_name}\\n(p=500, n_train={n_train}, {len(real_data_results)} models tested)', 
          fontsize=14, fontweight='bold')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Best Performance')
plt.legend()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
print("\n📈 Summary Statistics:")
print("-"*70)
baseline_names = list(baseline_models.keys())
baseline_errors = [relative_errors[m] for m in baseline_names if m in relative_errors]
rlt_names = [m for m in relative_errors.keys() if m.startswith('RLT-') and m != 'RLT-naive']
rlt_errors = [relative_errors[m] for m in rlt_names]

print(f"Baseline models (n={len(baseline_errors)}):")
print(f"  Best: {min(baseline_errors):.3f}")
print(f"  Worst: {max(baseline_errors):.3f}")
print(f"  Mean: {np.mean(baseline_errors):.3f}")

print(f"\nRLT variants (n={len(rlt_errors)}):")
print(f"  Best: {min(rlt_errors):.3f}")
print(f"  Worst: {max(rlt_errors):.3f}")
print(f"  Mean: {np.mean(rlt_errors):.3f}")

best_model_name = sorted_results[0][0]
best_model_error = real_data_results[best_model_name]
print(f"\n🏆 Overall Best: {best_model_name} ({metric_name}={best_model_error:.4f})")

print("\n✅ Real Data Comparison Complete!")
print("="*70)
